In [1]:
# 第十章作业
# 基于 data 目录下的数据训练 ChatGLM3 模型，使用 inference Notebook 对比微调前后的效果

import os

os.environ['HF_HOME'] = 'D:\MTIDE\.cache\huggingface'
os.environ['HF_HUB_CACHE'] = 'D:\MTIDE\.cache\huggingface\hub'

# 定义全局变量和参数
# model_name_or_path = 'THUDM/chatglm3-6b'  # 模型ID或本地路径
model_name_or_path = 'D:\MTIDE\code\AI\models\THUDM\chatglm3-6b'  # 模型ID或本地路径
train_data_path = 'data/zhouyi_dataset_20240118_163659.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None                     # 验证数据路径，如果没有则设置为None
seed = 8                                 # 随机种子
max_input_length = 512                    # 输入的最大长度
max_output_length = 1536                  # 输出的最大长度
lora_rank = 16                             # LoRA秩
lora_alpha = 32                           # LoRA alpha值
lora_dropout = 0.05                       # LoRA Dropout率
prompt_text = ''                          # 所有数据前的指令文本

In [2]:
# 数据处理

from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}

DatasetDict({
    train: Dataset({
        features: ['content', 'summary'],
        num_rows: 160
    })
})


Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [3]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False, 
    remove_columns=column_names
)

In [4]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

In [5]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:
    """
    用于处理批量数据的DataCollator，尤其是在使用 ChatGLM 模型时。

    该类负责将多个数据样本（tokenized input）合并为一个批量，并在必要时进行填充(padding)。

    属性:
    pad_token_id (int): 用于填充(padding)的token ID。
    max_length (int): 单个批量数据的最大长度限制。
    ignore_label_id (int): 在标签中用于填充的ID。
    """

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):
        """
        初始化DataCollator。

        参数:
        pad_token_id (int): 用于填充(padding)的token ID。
        max_length (int): 单个批量数据的最大长度限制。
        ignore_label_id (int): 在标签中用于填充的ID，默认为-100。
        """
        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:
        """
        处理批量数据。

        参数:
        batch_data (List[Dict[str, List]]): 包含多个样本的字典列表。

        返回:
        Dict[str, torch.Tensor]: 包含处理后的批量数据的字典。
        """
        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}

In [6]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

In [7]:
# 加载模型

from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True,
                                  revision='b098244')

model.supports_gradient_checkpointing = True  
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

In [9]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [10]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


In [11]:
# QLoRA 微调模型

import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

train_epochs = 3
output_dir = f"models/THUDM/chatglm3-6b-epoch{train_epochs}-{timestamp}"

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,                            # 输出目录
    per_device_train_batch_size=8,                     # 每个设备的训练批量大小
    gradient_accumulation_steps=1,                     # 梯度累积步数
    learning_rate=1e-3,                                # 学习率
    num_train_epochs=train_epochs,                     # 训练轮数
    lr_scheduler_type="linear",                        # 学习率调度器类型
    warmup_ratio=0.1,                                  # 预热比例
    logging_steps=1,                                 # 日志记录步数
    save_strategy="steps",                             # 模型保存策略
    save_steps=10,                                    # 模型保存步数
    optim="adamw_torch",                               # 优化器类型
    fp16=True,                                        # 是否使用混合精度训练
)

trainer = Trainer(
    model=qlora_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

C:\Users\mtide\miniconda3\envs\py311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
trainer.train()

wandb: Currently logged in as: eddiezhang (eddieorg). Use `wandb login --relogin` to force relogin


D:\MTIDE\.cache\huggingface\modules\transformers_modules\chatglm3-6b\modeling_chatglm.py:226: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(query_layer, key_layer, value_layer,


Step,Training Loss
1,4.526700
2,4.508800
3,4.366800
4,4.304000
5,4.166100
6,3.726400
7,3.620900
8,3.293000
9,3.190800
10,2.910600


C:\Users\mtide\miniconda3\envs\py311\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in D:\MTIDE\code\AI\models\THUDM\chatglm3-6b - will assume that the vocabulary was not modified.
  warnings.warn(
C:\Users\mtide\miniconda3\envs\py311\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in D:\MTIDE\code\AI\models\THUDM\chatglm3-6b - will assume that the vocabulary was not modified.
  warnings.warn(
C:\Users\mtide\miniconda3\envs\py311\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in D:\MTIDE\code\AI\models\THUDM\chatglm3-6b - will assume that the vocabulary was not modified.
  warnings.warn(
C:\Users\mtide\miniconda3\envs\py311\Lib\site-packages\peft\utils\save_and_load.py:154: UserWarning: Could not find a config file in D:\MTIDE\code\AI\models\THUDM\chatglm3-6b - will assume that the vocabulary was not modified.
  warnings.warn(
C:\Users\mtide\miniconda

TrainOutput(global_step=60, training_loss=0.9526841989175107, metrics={'train_runtime': 1801.181, 'train_samples_per_second': 0.266, 'train_steps_per_second': 0.033, 'total_flos': 4649502945607680.0, 'train_loss': 0.9526841989175107, 'epoch': 3.0})

In [ ]:
trainer.model.save_pretrained(output_dir)

In [1]:
# 模型推理

# 使用 QLoRA 微调后的 ChatGLM-6B
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
# model_name_or_path = 'THUDM/chatglm3-6b'
model_name_or_path = 'D:\MTIDE\code\AI\models\THUDM\chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

base_model.requires_grad_(False)
base_model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


In [4]:
# 使用原始 ChatGLM3-6B 模型
input_text = "解释下乾卦是什么？"
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

C:\Users\mtide\.cache\huggingface\modules\transformers_modules\chatglm3-6b\modeling_chatglm.py:226: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(query_layer, key_layer, value_layer,


乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、强、积极、刚毅、正义等含义。

乾卦的卦辞是“元、亨、利、贞”，其中“元”表示至高无上的道德原则，“亨”表示通，顺利，“利”表示有益，“贞”表示正。这些词语体现了乾卦所代表的正义、积极和刚毅的品质。

在《易经》中，乾卦的五行属性是“木”，与肝脏、筋和角相对应。同时，乾卦也与春季、东方、木element以及男性等有关。

乾卦的含义是，当面临困难或挑战时，我们应该保持积极的态度，勇敢地面对问题，寻求解决办法，并坚持正义的原则，最终达到成功。
讼卦是《易经》中的第五卦，也是八卦之一。讼卦由两个阳爻夹一个阴爻构成，象征着诉讼、争端、矛盾和困扰等含义。

讼卦的卦辞是“天ult、无乃、贞、好、元”，其中“天ult”表示天意，意为天意难以预测；“无乃”表示不是吗，“贞”表示正，“好”表示美好，“元”表示至高无上的道德原则。这些词语体现了讼卦所代表的矛盾、困扰和困扰中的正义和公正。

在《易经》中，讼卦的五行属性是“火”，与心脏、是小肠和南方相对应。同时，讼卦也与诉讼、争端、矛盾和困扰等有关。

讼卦的含义是，当面临矛盾和困扰时，我们应该保持冷静和理智，寻求和平解决办法，避免争端和诉讼。在处理问题时，我们应该遵循正义和公正的原则，以达到最终的美好结果。


In [8]:
# 使用微调后的 ChatGLM3-6B
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240417_125044"

peft_model_path = f"models/THUDM/chatglm3-6b-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [9]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [10]:
# 微调前后效果对比

base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个乾卦叠加而成，代表天卦。它象征着健立，刚健，自强不息的意义。乾卦的核心哲学是：天行健，君子自强不息。乾卦的解释是：健 spec', 'content': '\n动 1. 初六：吉，无不利。 spec\n\n动 2. 七五：吉 critical\n\n动 3. 九一：大吉，德高望检\n\n行动：君子观此卦象，应当自强不息，努力不懈，以达到化险为夷的目的。在事业上，应当勇敢自强，努力进取，不可止步不前。在经商中，应当自强不息，积极进取，止于至善，德高望检。在决策中，应当自强不息，勤勉努力，追求至善至美。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240417_125044）：
[gMASK] sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [11]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是由上卦乾（天）和下卦坎（水）组成的卦象，代表着诉讼和纷争。在这个卦象中，天代表着刚强，坎代表着柔和。因此，卦象中刚强与柔和相互对立，互不相让，导致了卦象中的矛盾和纷争。然而，虽然讼卦表象的是冲突和争端，但实际上，天道是公允的，只要时机成熟，就会得到解决。因此，应当等待时机，同时也要坚持正道，积极行动，才能化解争端，化解矛盾。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240417_125044）：
[gMASK] sop 周易中的讼卦是什么样子 讼卦是一个由上卦乾（天）和下卦坎（水）组成的卦象。它代表着诉讼和纠纷。在这个卦象中，天代表着刚健和健行，坎代表着柔顺和适应。因此，解决讼卦的问题需要既刚健又柔顺，同时也需要适应情况，才能获得美好的结果。

讼卦的核心哲学是：天行健，君子拟天行健；水自流，君子拟水自流。这意味着君子应当效法天行健的刚健，以及水自流般的柔顺。在解决纷争诉讼的问题时，需要双方都能理解和适应，才能获得美好的结果。

对于君子来说，解决讼卦的问题需要既有刚健的品德，又有柔顺的品性。在处理诉讼时，需要顺应天意，根据情况灵活应对。对于是非对错的问题，需要以德服人，用正义来解决。

讼卦虽然不是一个吉卦，但它提示了解决纷争诉讼的问题需要刚健和柔顺的结合。在处理诉讼时，需要适应情况，根据天意和道义来解决。


In [12]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '师卦是由坎卦（水）和坤卦（地）相叠而成的异卦。在周易中，这是唯一一种由两个卦相异的卦象，也是唯一一个由地卦和坎卦相异的卦象。地卦象征着 earth，坎卦象征着 stream，象征着连通之意，因此，师卦象征着 stream that connects，意味着地中有水，鱼跃泉出的景象。师卦预示着吉祥，但 also预示着警惕，因为地中虽然有水，但也有可能出现危险。师卦的核心在于君子观此卦象，从而得到启示，并采取相应的行动。', 'content': '\n师卦象征着吉祥，但 also需要注意风险。君子观此卦象，可以得到启示，并采取相应的行动。在事业方面，预示着吉祥，务必谨慎行事，小心谨慎地开展事业。在婚姻方面，象征着吉祥，务必小心谨慎地处理婚姻事务。在经商方面，也预示着吉祥，但务必小心谨慎行事，注意安全，同时，也需注意 financial危险。师卦的核心在于，必须小心的处理连通之事，但也需要勇敢地向前，勇敢迎接挑战。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240417_125044）：
[gMASK] sop 师卦是什么？ 师卦是由坎卦（水）和坤卦（地）相叠而成的异卦。在周易中，这是唯一一种由两个卦象组成的卦象，代表着军队的力量和军情的总指挥。师卦预示着吉祥无灾，但预示着君子谨慎行事，做好各项工作的安排，等待时机。

坎卦象征着水，代表着灵活、适应、灵活调整；坤卦象征着地，代表着稳定、安顺、稳定。两者相结合，意味着安全、吉祥、顺利，但也提示着前进勿忘观法（犹若行船），做好规划。

师卦的核心哲学是：时机成熟，行动之前要谋略、计划和行动要迅速、灵活适应，方能顺利成功。在事业、经商、求名、婚恋等方面的决策中，都需要谨慎行事，做好规划，等待时机，方能成功。
